In [1]:
import pandas as pd
import numpy as np

from statsmodels.tsa.stattools import adfuller, acf, pacf

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, ARIMA
from statsforecast.utils import AirPassengersDF

from sklearn.metrics import mean_squared_error, mean_absolute_error

import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp

/home/heylucasleao/forecasting/.venv/lib/python3.10/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore
/home/heylucasleao/forecasting/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
data = pd.read_csv(url, parse_dates=['Month'], index_col='Month')
series = data['Passengers']

# Plot using Plotly
fig = px.line(series, x=series.index, y=series.values,
              title='Airline Passengers Data (1949–1960)',
              labels={'Month': 'Date', 'Passengers': 'Number of Passengers'},
              template='plotly_white')

fig.update_layout(title_x=0.5)
fig.show()

In [3]:
# ADF Test (Original Series)
adf_original = adfuller(series)
print(f"ADF Statistic (original): {adf_original[0]:.4f}")
print(f"p-value (original): {adf_original[1]:.4f}")

# Differenced series
diff_series = series.diff().dropna()

# ADF Test (Differenced Series)
adf_diff = adfuller(diff_series)
print(f"ADF Statistic (differenced): {adf_diff[0]:.4f}")
print(f"p-value (differenced): {adf_diff[1]:.4f}")

ADF Statistic (original): 0.8154
p-value (original): 0.9919
ADF Statistic (differenced): -2.8293
p-value (differenced): 0.0542


In [4]:
# Differencing
diff_1 = series.diff().dropna()
diff_seasonal = series.diff(12).dropna()
diff_both = diff_1.diff(12).dropna()
n_diff = len(diff_both)

# ACF/PACF values
def create_acf_pacf_traces(data, nlags=30, color='blue'):
    n = len(data)
    conf = 1.96 / np.sqrt(n)
    acf_vals = acf(data, nlags=nlags)
    pacf_vals = pacf(data, nlags=nlags, method='yw')

    acf_bar = go.Bar(x=list(range(len(acf_vals))), y=acf_vals, marker_color=color)
    pacf_bar = go.Bar(x=list(range(len(pacf_vals))), y=pacf_vals, marker_color=color)

    band_upper = go.Scatter(x=list(range(nlags+1)), y=[conf]*(nlags+1),
                            mode='lines', line=dict(color='gray', dash='dash'), showlegend=False)
    band_lower = go.Scatter(x=list(range(nlags+1)), y=[-conf]*(nlags+1),
                            mode='lines', line=dict(color='gray', dash='dash'), showlegend=False)

    return acf_bar, pacf_bar, band_upper, band_lower

acf_orig, pacf_orig, conf_up_o, conf_lo_o = create_acf_pacf_traces(series, color='steelblue')
acf_1, pacf_1, conf_up_1, conf_lo_1 = create_acf_pacf_traces(diff_1, color='orange')
acf_both, pacf_both, conf_up_b, conf_lo_b = create_acf_pacf_traces(diff_both, color='purple')

# ADF Results
adf_results = {
    "Original": adfuller(series),
    "First-Order": adfuller(diff_1),
    "Seasonal (Lag 12)": adfuller(diff_seasonal.dropna()),
    "First + Seasonal": adfuller(diff_both.dropna())
}

adf_text = "<br>".join([f"<b>{k}</b>: ADF={v[0]:.4f}, p={v[1]:.4f}" for k, v in adf_results.items()])

# Create 4x3 layout
fig = sp.make_subplots(
    rows=4, cols=3,
    subplot_titles=[
        "Original Series", "ACF (Original)", "PACF (Original)",
        "First-Order Differenced", "ACF (First-Diff)", "PACF (First-Diff)",
        "First + Seasonal Differenced", "ACF (Full-Diff)", "PACF (Full-Diff)",
        "ADF Results Summary", "", ""
    ]
)

# Row 1
fig.add_trace(go.Scatter(x=series.index, y=series, mode='lines', name='Original'), row=1, col=1)
fig.add_trace(acf_orig, row=1, col=2)
fig.add_trace(pacf_orig, row=1, col=3)
fig.add_trace(conf_up_o, row=1, col=2)
fig.add_trace(conf_lo_o, row=1, col=2)
fig.add_trace(conf_up_o, row=1, col=3)
fig.add_trace(conf_lo_o, row=1, col=3)

# Row 2
fig.add_trace(go.Scatter(x=diff_1.index, y=diff_1, mode='lines', name='First Diff', line=dict(color='orange')), row=2, col=1)
fig.add_trace(acf_1, row=2, col=2)
fig.add_trace(pacf_1, row=2, col=3)
fig.add_trace(conf_up_1, row=2, col=2)
fig.add_trace(conf_lo_1, row=2, col=2)
fig.add_trace(conf_up_1, row=2, col=3)
fig.add_trace(conf_lo_1, row=2, col=3)

# Row 3
fig.add_trace(go.Scatter(x=diff_both.index, y=diff_both, mode='lines', name='Full Diff', line=dict(color='purple')), row=3, col=1)
fig.add_trace(acf_both, row=3, col=2)
fig.add_trace(pacf_both, row=3, col=3)
fig.add_trace(conf_up_b, row=3, col=2)
fig.add_trace(conf_lo_b, row=3, col=2)
fig.add_trace(conf_up_b, row=3, col=3)
fig.add_trace(conf_lo_b, row=3, col=3)

# Row 4 - ADF summary as annotation
fig.add_trace(go.Scatter(
    x=[0], y=[0], text=[adf_text],
    mode='text', showlegend=False
), row=4, col=1)

# Layout
fig.update_layout(
    title="ACF/PACF Across Differencing Stages with ADF Summary",
    height=1200,
    width=1300,
    showlegend=False
)

# Axis labels
for row in range(1, 4):
    fig.update_xaxes(title_text="Date", row=row, col=1)
    fig.update_yaxes(title_text="Value", row=row, col=1)
    fig.update_xaxes(title_text="Lag", row=row, col=2)
    fig.update_xaxes(title_text="Lag", row=row, col=3)
    fig.update_yaxes(title_text="ACF", row=row, col=2)
    fig.update_yaxes(title_text="PACF", row=row, col=3)

fig.update_xaxes(visible=False, row=4, col=1)
fig.update_yaxes(visible=False, row=4, col=1)

fig.show()

In [5]:
# df = AirPassengersDF
# sf = StatsForecast(
#     models=[AutoARIMA(season_length=12)],
#     freq='ME',
# )
# df

In [6]:
forecast_horizon = 12

# Convert Series to DataFrame with 'ds' and 'y' columns
df = series.reset_index()
df.columns = ['ds', 'y']
df['unique_id'] = 1

# Split data
train_df = df.iloc[:-forecast_horizon]
test_df = df.iloc[-forecast_horizon:]

print(f"Training data size: {len(train_df)}")
print(f"Test data size (forecast horizon): {len(test_df)}")

Training data size: 132
Test data size (forecast horizon): 12


In [7]:
# Initialize AutoARIMA model
auto_model = AutoARIMA(season_length=12)

# Initialize StatsForecast engine
sf_auto = StatsForecast(models=[auto_model], freq='MS')

# Fit the model (corrected: pass df, not series)
sf_auto.fit(df=train_df, id_col='unique_id', time_col='ds', target_col='y')

# Generate forecast
auto_forecast_df = sf_auto.predict(h=12)

# Extract parameters
fitted_auto_arima_object = sf_auto.fitted_[0, 0]
fitted_model_parameters_dict = fitted_auto_arima_object.model_
# Assuming fitted_model_parameters_dict has been correctly obtained as above

print(f"ARIMA Order (p,d,q)(P,D,Q)s: {fitted_model_parameters_dict.get('arma')}")
print(f"Coefficients: {fitted_model_parameters_dict.get('coef')}")
aic = fitted_model_parameters_dict.get('aic')
bic = fitted_model_parameters_dict.get('bic')
aicc = fitted_model_parameters_dict.get('aicc')
print(f"AIC: {aic}, BIC: {bic}, AICC: {aicc}")
print(f"Residuals (first 5): {fitted_model_parameters_dict.get('residuals')[:5]}")

ARIMA Order (p,d,q)(P,D,Q)s: (1, 0, 0, 0, 12, 1, 1)
Coefficients: {'ar1': np.float64(-0.24476352132880558)}
AIC: 899.9024574640343, BIC: 905.4607044502574, AICC: 900.0059057398964
Residuals (first 5): [0.06466322 0.03356584 0.03380614 0.02255184 0.01075386]


In [8]:
# 2. Define and Fit Manually Specified SARIMA Model
sarima_manual_model = ARIMA(
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1),
    season_length=12,
)


# Initialize StatsForecast engine
sf_manual = StatsForecast(models=[sarima_manual_model], freq='MS')

# Fit the model (corrected: pass df, not series)
sf_manual.fit(df=train_df, id_col='unique_id', time_col='ds', target_col='y')


print("\nFitting manually specified SARIMA(1,1,1)(1,1,1)_12 model on training data...")
print("Manual model fitting complete.")

# 3. Generate Forecasts for the Test Period
print("\nGenerating forecasts for the test period...")
manual_forecast_df = sf_manual.predict(h=forecast_horizon)
auto_forecast_df = sf_auto .predict(h=forecast_horizon)

# 5. Extract True and Predicted Values
y_true = test_df['y'].values
y_pred_manual = manual_forecast_df['ARIMA'].values
y_pred_auto = auto_forecast_df['AutoARIMA'].values

# 6. Calculate RMSE and MAE
rmse_manual = np.sqrt(mean_squared_error(y_true, y_pred_manual))
mae_manual = mean_absolute_error(y_true, y_pred_manual)

rmse_auto = np.sqrt(mean_squared_error(y_true, y_pred_auto))
mae_auto = mean_absolute_error(y_true, y_pred_auto)

print("\n--- Forecast Evaluation Metrics ---")
print(f"Manual SARIMA(1,1,1)(1,1,1)_12:")
print(f"  RMSE: {rmse_manual:.3f}")
print(f"  MAE: {mae_manual:.3f}")

print(f"AutoARIMA Selected Model:")
print(f"  RMSE: {rmse_auto:.3f}")
print(f"  MAE: {mae_auto:.3f}")


# Assuming sf_manual and sf_auto have been fitted as in the full example code.

# --- Display fitted model parameters for the Manual SARIMA model ---
print("\n--- Fitted Manual SARIMA Model Parameters ---")
fitted_manual_params = sf_manual.fitted_[0, 0].model_
print(f"ARIMA Order (p,d,q)(P,D,Q)s: {fitted_manual_params.get('arma')}")
aic_manual = fitted_manual_params.get('aic')
bic_manual = fitted_manual_params.get('bic')
aicc_manual = fitted_manual_params.get('aicc')
print(f"AIC: {aic_manual:.4f}, BIC: {bic_manual:.4f}, AICC: {aicc_manual:.4f}")

# --- Display fitted model parameters for the AutoARIMA model ---
print("\n--- Fitted AutoARIMA Model Parameters ---")
fitted_auto_params = sf_auto.fitted_[0, 0].model_
print(f"ARIMA Order (p,d,q)(P,D,Q)s: {fitted_auto_params.get('arma')}")
aic_auto = fitted_auto_params.get('aic')
bic_auto = fitted_auto_params.get('bic')
aicc_auto = fitted_auto_params.get('aicc')
print(f"AIC: {aic_auto:.4f}, BIC: {bic_auto:.4f}, AICC: {aicc_auto:.4f}")


Fitting manually specified SARIMA(1,1,1)(1,1,1)_12 model on training data...
Manual model fitting complete.

Generating forecasts for the test period...

--- Forecast Evaluation Metrics ---
Manual SARIMA(1,1,1)(1,1,1)_12:
  RMSE: 21.464
  MAE: 16.566
AutoARIMA Selected Model:
  RMSE: 23.919
  MAE: 18.516

--- Fitted Manual SARIMA Model Parameters ---
ARIMA Order (p,d,q)(P,D,Q)s: (1, 1, 1, 1, 12, 1, 1)
AIC: 903.1681, BIC: 917.0637, AICC: 903.6991

--- Fitted AutoARIMA Model Parameters ---
ARIMA Order (p,d,q)(P,D,Q)s: (1, 0, 0, 0, 12, 1, 1)
AIC: 899.9025, BIC: 905.4607, AICC: 900.0059
